In [3]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [4]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [5]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [6]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    491     491 

## Models

In [7]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [8]:

model_monmlp <- train(x=TrainData, y=TrainClasses, 
               method = "monmlp") 

** Ensemble 1 
Complex eigenvalues found for method = BFGS 
coefficients for function value 0.3233001  :
  [1]  1.859767e+02  5.944360e+02 -2.046715e+00  1.772597e+02  1.518588e+02
  [6] -4.053607e+02 -5.149329e+02  3.600633e+01 -3.599583e+01  1.432298e+02
 [11] -9.887372e+01  8.896495e+00 -3.603721e+02  3.393949e+02 -1.056244e+02
 [16]  1.057234e+02 -2.386611e+01  1.522829e+02 -7.961785e+01 -4.187002e+01
 [21]  1.763047e+02 -1.763472e+02 -7.187687e+01 -4.787875e+01  1.355686e+02
 [26]  2.442024e+02 -6.930941e+01 -9.959654e+01  9.961702e+01 -2.899423e+01
 [31]  2.894626e+01 -1.476774e+02  1.476869e+02 -1.616197e+02  3.998185e+02
 [36] -1.166589e+02 -3.576520e+02  2.564030e+02  1.599683e+02 -1.600067e+02
 [41] -1.761401e+02  4.310693e+02  7.724823e+02 -3.262501e+02 -3.544784e+02
 [46]  7.746322e+02  7.625368e+01 -1.327578e+02 -4.206790e+02 -2.775178e+02
 [51] -4.552492e+02  1.429042e+00 -6.277453e+02 -5.932216e+02  8.874173e+01
 [56] -3.584165e+01  6.294745e+01 -2.095842e+02 -7.463089e+

In [9]:

model_mlpW <- train(x=TrainData, y=TrainClasses, 
               method = "mlpWeightDecay") 

## Results

In [10]:
print(model_monmlp)
plot(model_monmlp,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Monotone Multi-Layer Perceptron Neural Network 

982 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 982, 982, 982, 982, 982, 982, ... 
Resampling results across tuning parameters:

  hidden1  Accuracy   Kappa    
  1        0.6972950  0.3951318
  3        0.7042040  0.4090431
  5        0.6710723  0.3427657

Tuning parameter 'n.ensemble' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were hidden1 = 3 and n.ensemble = 1.


In [11]:
print(model_mlpW)
plot(model_mlpW,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Multi-Layer Perceptron 

982 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 982, 982, 982, 982, 982, 982, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.7104063  0.4218583
  1     1e-04  0.7074791  0.4157873
  1     1e-01  0.4998277  0.0000000
  3     0e+00  0.7193231  0.4390465
  3     1e-04  0.7228148  0.4467490
  3     1e-01  0.5037840  0.0000000
  5     0e+00  0.7227042  0.4458700
  5     1e-04  0.7211682  0.4430927
  5     1e-01  0.5059108  0.0000000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 3 and decay = 1e-04.


In [12]:
# collect resamples
results <- resamples(list(monmlp=model_monmlp,  mlpWeightDecay=model_mlpW))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: monmlp, mlpWeightDecay 
Number of resamples: 25 

Accuracy 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.6504065 0.6906077 0.7037037 0.7042040 0.7130919 0.7608069    0
mlpWeightDecay 0.6702413 0.7063712 0.7242340 0.7228148 0.7434555 0.7741935    0

Kappa 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.3054092 0.3853568 0.4104371 0.4090431 0.4265443 0.5214588    0
mlpWeightDecay 0.3541171 0.4126477 0.4486092 0.4467490 0.4860094 0.5484915    0


## Prediction

In [13]:
pred_mlpW <- predict(model_mlpW,TestData)
pred_mlpW.prob <- predict(model_mlpW,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_mlpW) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     129      18
   NotFast  149     125
                                          
               Accuracy : 0.6033          
                 95% CI : (0.5548, 0.6504)
    No Information Rate : 0.6603          
    P-Value [Acc > NIR] : 0.9937          
                                          
                  Kappa : 0.2766          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.4640          
            Specificity : 0.8741          
         Pos Pred Value : 0.8776          
         Neg Pred Value : 0.4562          
             Prevalence : 0.6603          
         Detection Rate : 0.3064          
   Detection Prevalence : 0.3492          
      Balanced Accuracy : 0.6691          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.4640288            0.8741259            0.8775510 
      Neg Pred Value            Precision               Recall 
           0.4562044            0.8775510            0.4640288 
                  F1           Prevalence       Detection Rate 
           0.6070588            0.6603325            0.3064133 
Detection Prevalence    Balanced Accuracy 
           0.3491686            0.6690773

In [16]:
roc_obj <- multiclass.roc(TestClasses, as.numeric(pred_mlpW))
roc_obj


Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_mlpW))

Data: as.numeric(pred_mlpW) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6669

In [19]:
pred_mlpW.prob

,fast,NotFast
16,0.51724106,0.4827590
33,0.88768989,0.1123101
78,0.59314829,0.4068517
79,0.84841371,0.1515863
83,0.86172998,0.1382701
120,0.87880933,0.1211907
122,0.87145001,0.1285500
123,0.58895737,0.4110427
125,0.36061838,0.6393816
132,0.38504550,0.6149545
